<a href="https://colab.research.google.com/github/edchan7-sama/sama-api-colabs/blob/main/Task_Deliveries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Settings

In [ ]:
#@title Step 1 - Set Project Settings { run: "auto", display-mode: "form" }
project_id = "" #@param {type:"string"}
client_access_key = "" #@param {type:"string"}
delivery_datetime = "2021-02-02" #@param {type:"date"}
delivery_datetime += "T00:00:00Z"

# Get Deliveries 

### Step 2 - Get Deliveries as JSON


In [ ]:
import requests 
from google.colab import files
import json


# Query URL
url = f"https://api.sama.com/api/v2/projects/{project_id}/tasks/delivered.json"

page = 1; page_size = 1000; tasks = []

while True:
  try:
    querystring = {"from":delivery_datetime,
                  "page":page,
                  "page_size":page_size,
                  "access_key":client_access_key}

    headers = {"Accept": "application/json"}

    # Make a GET request to the URL
    response = requests.request("GET", url, headers=headers, params=querystring)

    # Parse `response.text` into JSON
    payload = response.json()

    if response.status_code == 200:
      if payload.get("tasks"):
        tasks += payload.get("tasks")
        page += 1
      elif page == 1:
        raise Exception(f"No task deliveries for project {project_id} from {delivery_datetime}")
      else:
        raise Exception(f"Task deliveries available for project {project_id} from {delivery_datetime}")
    else:
      raise Exception(f"Error: Invalid response code, {response.status_code} : '{response.reason}'")
  except Exception as e:
    print(str(e))
    break

if tasks:
  with open(f"{project_id}_{delivery_datetime}.json", "w") as f:
    json.dump(tasks, f, indent=4)
  files.download(f"{project_id}_{delivery_datetime}.json")
  print(f"Downloaded: {project_id}_{delivery_datetime}.json")


Task deliveries available for project 7536 from 2021-02-02T00:00:00Z


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded: 7536_2021-02-02T00:00:00Z.json


### Step 3 - Optional - Save as CSV

In [ ]:
!pip install flatten_json --quiet
import pandas as pd
from flatten_json import flatten

# answer objects are nested JSONs, we want to keep them that way. 

try:
  if not tasks:
    raise Exception(f"No task deliveries for project {project_id} from {delivery_datetime}")

  # create a DataFrame from the task objects, but ignore answers as they should be kept as a dict 
  dic_flattened = [flatten(d, root_keys_to_ignore={'answers'}) for d in tasks]
  df = pd.DataFrame(dic_flattened)

  # create a DataFrame of just the answer objects, but convert to json
  answers = []
  for d in tasks:
      a = { key: json.dumps(d.get('answers')[key]) for key in d.get('answers') }
      a['id'] = d.get('id')
      answers.append(a)
  df_answers = pd.DataFrame(answers)

  csv_name = f"{project_id}_{delivery_datetime}.csv"
  # merge DataFrames and write to csv
  pd.merge(df, df_answers, on='id').to_csv(csv_name)
  files.download(csv_name)

except NameError:
  print("Run Get Delivered Tasks First")
except Exception as e:
  print(str(e))


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>